<a href="https://colab.research.google.com/github/bkglobal/zoomcamp_data_engineering/blob/main/DTC_data_extraction_and_ingestion_completed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [10]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)
sum = 0
for sqrt_value in generator:
    sum = sum + sqrt_value

print(f"Sum of values are : {sum}")

limit = 13
generator = square_root_generator(limit)
for sqrt_value in generator:
    print(sqrt_value)

Sum of values are : 8.382332347441762
1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [36]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [37]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [40]:
# to do: homework :)

import dlt

# Laod the first generator..
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')

people_1_append_info = generators_pipeline.run(people_1(),
										table_name="people_1_append",
										write_disposition="replace")


people_1_append_info = generators_pipeline.run(people_2(),
										table_name="people_1_append",
										write_disposition="append")


In [41]:
# Question 3. Append the 2 generators. After correctly appending the data, calculate the sum of all ages of people.

import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")
# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))


people_1_append_table = conn.sql("SELECT * FROM people_1_append").df()

sum_of_ages_people_1_append = people_1_append_table.age.sum()

print("Sum of ages of first generator people : ", sum_of_ages_people_1_append )

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people_1            │
│ people_1_append     │
└─────────────────────┘

Sum of ages of first generator people :  353


In [42]:
# Question 4. Merge the 2 generators using the ID column. Calculate the sum of ages of all the people loaded as described above.

import dlt

# Laod the first generator..
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')

people_1_info = generators_pipeline.run(people_1(),
										table_name="people_1",
										write_disposition="replace",
                                        primary_key="ID")


people_1_info = generators_pipeline.run(people_2(),
										table_name="people_1",
										write_disposition="merge",
                                        primary_key="ID")
print(people_1_info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.38 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708017884.7038982 is LOADED and contains no failed jobs


In [44]:

import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")
# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))


people_1_table = conn.sql("SELECT * FROM people_1").df()

sum_of_ages_people_1 = people_1_table.age.sum()

print("Sum of ages of generator people : ", sum_of_ages_people_1 )


Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people_1            │
│ people_1_append     │
└─────────────────────┘

Sum of ages of generator people :  266


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX